In [1]:

import requests
import requests.auth
import regex as re
from os.path  import basename
import random
import string
import json

In [ ]:
client_auth = requests.auth.HTTPBasicAuth('AlFmsJweahucqYg4PwnQhQ', 'TiGAXDkqpxFKi9OYqTJiodHMGVt3Ew')
post_data = {"grant_type": "password", "username": "RedditorInCh1ef", "password": 'Redd1tPassw0rd'}
headers_auth = {"User-Agent": "ChangeMeClient/0.1 by RedditorInCh1ef"}
response_auth = requests.post("https://www.reddit.com/api/v1/access_token", auth=client_auth, data=post_data, headers=headers_auth)
auth_response = response_auth.json()

In [ ]:
token = auth_response['access_token']
auth = "bearer " + token
reddit_headers = {"Authorization": auth, "User-Agent": "testing still"}
response = requests.get("https://oauth.reddit.com/r/photoshoprequest/hot", headers=reddit_headers)

In [5]:
def useable_link(i):
    """
    helper for get_links_from_post()
    """
    test = ['jpg','jpeg','png','imgur']
    for j in test:
        if j in i:
            return True
    return False

def get_links_from_post(postid,reddit_headers):
    """
    Takes a post id and returns the possible img links
    """
    print(postid)
    comments = requests.get(f'http://oauth.reddit.com/r/photoshoprequest/comments/{postid}', headers=reddit_headers)
    c = comments.json()
    try:
        post_comments = [x['data']['body'] for x in c[1]['data']['children']] ##c[0] is post data it appears
        reg_link_find = 'https.+?(?=[)\n\]])'
        results = list(set([j for i in post_comments for j in re.findall(reg_link_find,i)]))
        results = [i for i in results if 'paypal' not in i] #no tipjar links
        results = [i for i in results if useable_link(i) ] #good links
        results = [i.replace('https://imgur.com/','https://i.imgur.com/').strip() for i in results]  #fix the imgur links these need file ext.
        i_links = [i for i in links if 'imgur' in i]
        ii_links = [i.replace('imgur','i.imgur') for i in i_links]
        i_links = i_links + ii_links
        png_links = [i+'.png' for i in i_links if 'imgur' in i]
        jpg_links = [i+'.jpg' for i in i_links if 'imgur' in i]
        jpeg_links = [i+'.jpeg' for i in i_links if 'imgur' in i]
        i_links = i_links + png_links + jpg_links + jpeg_links
        
        return results + i_links
    except:
        return []
    
def download_image(img_url,length_of_file = 20):
    
    try:
        res=requests.get(img_url,allow_redirects = True,headers= {'Content-type': 'content_type_value'})
        response = requests.get(img_url)
        ext = response.headers['Content-Type'].split('/')[1]
        img_bytes= response.content # download bytes for a image
        print(ext)
        rand_string = get_random_string(length_of_file)
        file_name = f"{rand_string}.{ext}"
        if ext in ('jpeg','png'):
            with open(file_name,"wb") as img_file:
                img_file.write(img_bytes)
            return file_name
        else:
            print('here')
            pass
    except Exception as e:
        pass

def save_and_document(pre_img_url,post_image_urls,caption):
    """
    Takes a candidate pre_img_url and the resultant list from get_links_from_post
    saves the pre img if there is at least one output img.  saves files (makes sure they are unique)
    and adds to the metadata file.
    """
    output_image_list = []
    #input_image = [] ## this will be a list, of the ones that work.
    
    for link_candidate in post_image_urls:
        file_name = download_image(link_candidate)
        if file_name is not None:
            output_image_list.append(file_name)
    
    if len(output_image_list) > 0:
        input_image_file = download_image(pre_img_url,length_of_file = 20)
    
        if input_image_file is not None:
            if len(input_image_file) > 0:
                for output_image in output_image_list:
                    dictionary = {"output_image": output_image, "input_image":input_image_file, "text":caption}
                    json_object = json.dumps(dictionary, indent=4)
                    with open("metadata.jsonl", "a") as outfile:
                        outfile.write(json_object)

In [14]:
before = 1747782867
after = before - 500_000
request_url = f'https://api.pushshift.io/reddit/search/submission/?subreddit=PhotoshopRequest&size=500&num_comments=>50&over_18=false&before={before}&after={after}'

In [15]:
request_url

'https://api.pushshift.io/reddit/search/submission/?subreddit=PhotoshopRequest&size=500&num_comments=>50&over_18=false&before=1747782867&after=1747282867'

In [16]:
response = requests.get(request_url)
 
# print response
#print(response)
 
# print json content


In [17]:
res = response.json()

JSONDecodeError: [Errno Expecting value] <!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->
<head>


<title>api.pushshift.io | 522: Connection timed out</title>
<meta charset="UTF-8" />
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<meta http-equiv="X-UA-Compatible" content="IE=Edge" />
<meta name="robots" content="noindex, nofollow" />
<meta name="viewport" content="width=device-width,initial-scale=1" />
<link rel="stylesheet" id="cf_styles-css" href="/cdn-cgi/styles/main.css" />


</head>
<body>
<div id="cf-wrapper">

    

    <div id="cf-error-details" class="p-0">
        <header class="mx-auto pt-10 lg:pt-6 lg:px-8 w-240 lg:w-full mb-8">
            <h1 class="inline-block sm:block sm:mb-2 font-li

In [ ]:
links = get_links_from_post('tc8zvt',reddit_headers)

In [ ]:
link = 'https://i.imgur.com/CbJ7E5D.jpg'
cap = 'test'
save_and_document(link,['https://i.imgur.com/7Ycd0Lw.jpeg','https://i.imgur.com/CbJ7E5D.jpg'],cap)


In [ ]:
dates = []
for i in res['data']:
    print(i['title'])
    print(i['url'])
    print(i['created_utc'])
    dates.append(i['created_utc'])
    print(i['id'])
print(min(dates))

In [ ]:
with open("nextbeforedate.txt", "w") as outfile:
    outfile.write(str(after))